# Gensim

Gensim es una librería open source usada para el modelado de Topics y el procesado de lenguaje natural. Principalmente se usa para extraer topics semanticos de los documentos, permitiendo el uso de grandes colecciones de texto.

Tambien proporciona implementaciones multicore, de forma que sea posible paralelizar las tareas.

Es conveniente definir algunos terminos que se usaran a lo largo de este notebook:

- **Corpus:** Una coleccion de documentos de texto
- **Vector:** Forma númerica de representar texto
- **Modelo:** Algoritmo usado para obtener la representación del texto
- **Modelado de Topics:** Es la herramienta de mineria de texto, usada para extraer topics semanticos de documentos.
- **Topic:** Un repetitivo conjunto de palabras que ocurren juntas con cierta frecuencia.

## Lectura de datos

In [ ]:
import gensim.downloader as api
from pprint import pprint

In [ ]:
# Obtener los dataset disponibles de el api de gensim
info_datasets = api.info()
pprint(info_datasets)

{'corpora': {'20-newsgroups': {'checksum': 'c92fd4f6640a86d5ba89eaad818a9891',
                               'description': 'The notorious collection of '
                                              'approximately 20,000 newsgroup '
                                              'posts, partitioned (nearly) '
                                              'evenly across 20 different '
                                              'newsgroups.',
                               'fields': {'data': '',
                                          'id': 'original id inferred from '
                                                'folder name',
                                          'set': 'marker of original split '
                                                 "(possible values 'train' and "
                                                 "'test')",
                                          'topic': 'name of topic (20 variant '
                                                   'of pos

In [ ]:
# Obtener información de un dataset concreto, en este caso datos de una pagina de la wikipedia
dataset_info = api.info("text8")
pprint(dataset_info )

{'checksum': '68799af40b6bda07dfa47a32612e5364',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used '
                'for testing purposes; see wiki-english-* for proper full '
                'Wikipedia datasets.',
 'file_name': 'text8.gz',
 'file_size': 33182058,
 'license': 'not found',
 'num_records': 1701,
 'parts': 1,
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'record_format': 'list of str (tokens)'}


In [ ]:
# cargar los datos del dataset "text8"
from gensim.models import Word2Vec
dataset = api.load("text8")

[==================================================] 100.0% 31.6/31.6MB downloaded


In [ ]:
# Funcion para extraer el texto en crudo de un dataset de Gensim
def get_corpus_from_dataset(dataset):
  corpus =[]
  for sentence in dataset:  # iterate over all wiki script
    txt_sentence = (" ".join(sentence))
    corpus.append(txt_sentence)
  return ".\n".join(corpus)

In [ ]:
# Cargar un modelo preentrenado, en este caso a partir de noticias de Google, con vectores de 300 dimensiones
word2vec_model = api.load('word2vec-google-news-300')

[===========================-----------------------] 55.5% 922.7/1662.8MB downloaded

## Preparación de datos

### Tokenización

In [ ]:
import gensim
import os
from gensim.utils import simple_preprocess # simple_preprocess permite obtener una lista de tokens normalizados a partir de un texto.

corpus = get_corpus_from_dataset(dataset) # Obtenemos el texto en crudo usando la función que antes creamos (get_corpus_from_dataset)

# preprocess the file to get a list of tokens
tokenized =[]
for sentence in corpus.split('.'): # Extraemos las frases, con la aparición de puntos, e iteramos por ellas
  # the simple_preprocess function returns a list of each sentence
  tokenized.append(simple_preprocess(sentence, deacc = True)) # Extraemos los token de cada frase y los metemos en la lista tokenized. El parámeto deacc, normaliza el texto quitando los acentos


In [ ]:
print(tokenized[:10]) # Pintamos la tokenización de las 1o primeras frases

### Ceación del Diccionario

Ahora partinendo de la tokenización, podemos usar la función Dictionay, dentro del paquete corpus, para generar el diccionario de terminos

In [ ]:
from gensim import corpora
# Estraemos los tokens a el dicionario
my_dictionary = corpora.Dictionary(tokenized)
print(my_dictionary)

### Guardar el diccionario en disco


Accedemos a drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
PATH_MODELOS = '/content/drive/MyDrive/Colab Notebooks/nlp_introduction_course/models'
DICTIONARY_NAME = 'my_dictionary.dict'

Mounted at /content/drive


Guardamos el diccionario, llamando al metodo save con la ruta

In [ ]:
# save your dictionary to disk
my_dictionary.save(f'{PATH_MODELOS}/{DICTIONARY_NAME}')

Leemos el diccionario

In [ ]:
# load back
load_dict = corpora.Dictionary.load(f'{PATH_MODELOS}/{DICTIONARY_NAME}')

In [ ]:
print(load_dict)

Tambien podemos guardarlo como un fichero de texto

In [ ]:
# save your dictionary as text file
from gensim.test.utils import get_tmpfile
tmp_fname = get_tmpfile(f'{PATH_MODELOS}/dictionary')
my_dictionary.save_as_text(tmp_fname)

Y ahora como antes, podemos leearlo

In [ ]:
# load your dictionary text file
load_dict = corpora.Dictionary.load_from_text(tmp_fname)

In [ ]:
print(load_dict)

### Crear BOW desde el corpus


Para ello usamos la funcion **doc2bow( )**, que cuenta el número de ocurrencias de cada palabra 


In [ ]:
# converting to a bag of word corpus
BoW_corpus =[my_dictionary.doc2bow(doc, allow_update = True) for doc in tokenized]

In [ ]:
print(BoW_corpus[:10])

Como antes lo guardamos en el disco


In [ ]:
from gensim.corpora import MmCorpus
from gensim.test.utils import get_tmpfile
 
output_fname = get_tmpfile(f'{PATH_MODELOS}/BoW_corpus.mm')
 
# save corpus to disk
MmCorpus.serialize(output_fname, BoW_corpus)

Y cuando queramos podemos cargarlo desde el disco

In [ ]:
# load corpus
load_corpus = MmCorpus(output_fname)

### Crear una matriz TF-IDF con Gensim

Estos modelos son muy usados para determinar la importancia de las palabras en el corpus

In [ ]:
from gensim import models
import numpy as np
 
# Word weight in Bag of Words corpus
word_weight =[] # Lista de frecuencias, para cada palabra, palabra --> frecuencia
for doc in BoW_corpus: # Leemos la bolsa de palabras
  for id, freq in doc: # Recuperamos el id de la palabra y la frecuencia que aparece
    word_weight.append([my_dictionary[id], freq]) # Lo insertamos en la lista

In [ ]:
print(word_weight[:10])

Ahora vamos a usar el algoritmo TF-IDF con Gensim

In [ ]:
# create TF-IDF model
tfIdf = models.TfidfModel(BoW_corpus) # Creamos una instancia del Modelo TF-IDF, pasando la bolsa de palabras, e indicadon el parametro smartirs donde indicamos como queremos que se calcularlo (ver documentación)
 
# TF-IDF Word Weight
weight_tfidf =[]
for doc in tfIdf[BoW_corpus]:
  for id, freq in doc:
    weight_tfidf.append([my_dictionary[id], np.around(freq, decimals = 3)]) 

In [ ]:
weight_tfidf.sort(key=lambda x:x[1], reverse=True)

In [ ]:
weight_tfidf[:10]

Podemos ver que los topics que los terminos que parecen tener mas relevancia, parece que hacen mención a nombres propios, por ejemplo poe, dylan, gibraltar....

### Creación de Bigramas o Trigramas

- Bigrama: Grupo de 2 palabras
- Trigrama: Grupo de 3 palabras

Vamos a calcular los bigramas, usando el modelo Phrases, con 

In [ ]:
import gensim.downloader as api
from gensim.models.phrases import Phrases
 
# load the text8 dataset
dataset = api.load("text8")
 
# extract a list of words from the dataset
data =[] # Lista de palabras
for word in dataset:
  data.append(word)
               
# Bigram using Phraser Model             
bigram_model = Phrases(data, min_count = 3, threshold = 10) # min_count: numero minimo de frecuencia para contabilizar como bigrama. threshold: Puntuación minima para formar bigrama
 
print(bigram_model[data[0]]) 

In [ ]:
print(list(filter(lambda x: '_' in x, bigram_model[data[0]])) ) # Imprimo los bigramas, filtrando las palabras

Para calcular trigramas, podemos usar la salida anterior de los bigramas


In [ ]:
# Trigram using Phraser Model
trigram_model = Phrases(bigram_model[data], threshold = 10)


In [ ]:
print(list(filter(lambda x: x.count("_") == 2, trigram_model[data[0]])) ) # Imprimo los bigramas, filtrando las palabras

### Crear un modelo Word2Vec

Vamos a realizar un entrenamiento para poder generar una conversión a vectores de las palabras del corpus

In [ ]:
import gensim.downloader as api
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec
 
# leer datos de text8
dataset = api.load("text8")
 
# extraccion de lista de palabras
data =[]
for word in dataset:
  data.append(word)
 
# We will split the data into two parts
data_1 = data[:1200]   # 1200 instancias para entrenamiento
data_2 = data[1200:]   # 1200 para actualizcion del modelo
 
# Entrenar el modelo word2vec
w2v_model = Word2Vec(data_1, min_count = 0, workers = cpu_count())
 
# Mostrar el vector para la palabra "time"
print(w2v_model['time'])

Podemos testear el modelo, buscando la palabra mas similar a "time"

In [ ]:
# similar words to the word "time"
print(w2v_model.most_similar('time'))

Salvamos el modelo en disco

In [ ]:
# save your model
w2v_model.save(f'{PATH_MODELOS}/Word2VecModel-text8')

Y cuando lo necesitemso, tambien podemos leerlo de disco

In [ ]:
# load your model
model = Word2Vec.load(f'{PATH_MODELOS}/Word2VecModel-text8')

### Actualizar el modelo Word2Vec

Tambien podemos actualizar el modelo

Esto es muy util, para usar un modelo ya entrenado, y actualizarlo con texto especifico del dominio de nuestro problema, por ejemplo podriamos usar uno de caracter general, y mejorarlo con textos juridicos, para comprender mejor el idioma en ese contexto

Para el ejemplo, simplemente vamos a actualizarlo con los datos que separamos den data_2 anteriormente

In [ ]:
# build model vocabulary from a sequence of sentences
w2v_model.build_vocab(data_2, update = True)

Y entrenamos los vectores de palabras

In [ ]:
# train word vectors
w2v_model.train(data_2, total_examples = w2v_model.corpus_count, epochs = w2v_model.iter)

Y para el ejemplo volvemos a msotrar el vector de la palabra time, y podemos ver que ha cambiado

In [ ]:
print(w2v_model['time'])

### Doc2Vec

Esto es util para extraer vectores para documentos completos, o partes de documentos. Esto es muy util, para comparar documentos enteros o partes, por que siempre podemos calcular la similitud entre dos textos.

In [ ]:
import gensim
import gensim.downloader as api
from gensim.models import doc2vec
 
# Cargamos el dataset text8
dataset = api.load("text8") 
data =[] # Lista de listas
for w in dataset:
  data.append(w) 

[==================================================] 100.0% 31.6/31.6MB downloaded


Creamos una funcion para obtener una lista de documentos tageados, que es lo que servira de entrada al modelo

In [ ]:
# To train the model we need a list of tagged documents
def tagged_document(list_of_ListOfWords): # Recibimos la lista de palabras
  for x, ListOfWords in enumerate(list_of_ListOfWords): # Iteramos por la lista de palabras obteniendo la lista de palabras y un contador
    yield doc2vec.TaggedDocument(ListOfWords, [x]) # Creamos las anotaciones de documentos con la lista -> tad(id)

Llamamos a la función y lo ponemos en forma de lista

In [ ]:
# training data
data_train = list(tagged_document(data))

Mostramos el primer item de los datos tageados

In [ ]:
# print trained dataset
print(data_train[:1])

[TaggedDocument(words=['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing', 'interpretations', 'of', 'what', 'this', 'means', 'anarchism', 'also', 'refers'

Inicializamos el modelo

In [ ]:
# Initialize the model
d2v_model = doc2vec.Doc2Vec(vector_size = 40, min_count = 2, epochs = 30)

Le añadimos el vocabulario tageado que obtuvimos antes

In [ ]:
# build the vocabulary
d2v_model.build_vocab(data_train)

Y entrenamos el modelo

In [ ]:
# Train Doc2Vec model
d2v_model.train(data_train, total_examples = d2v_model.corpus_count, epochs = d2v_model.epochs, report_delay=10)

Vamos a guardar el modelo

In [ ]:
# save your model
d2v_model.save(f'{PATH_MODELOS}/Doc2VecModel-text8')

Y lo podemos volver a cargar

In [ ]:
# load your model
d2v_model = doc2vec.Doc2Vec.load(f'{PATH_MODELOS}/Doc2VecModel-text8')

Y por último vamos a visualizar un vector con el modelo entrenado

In [ ]:
# Analyzing the output
Analyze = d2v_model.infer_vector(['anarchism', 'against', 'against', 'working'])
print(Analyze)

[ 0.03813346  0.44497824  0.26099262  0.19917288  0.28539658 -0.49994126
  0.07968773  0.18806046  0.02574098 -0.07822325 -0.32427764  0.24641202
  0.04224819 -0.119422    0.17480847  0.08879094 -0.1247227   0.15645857
  0.14161716 -0.06526573  0.24324338 -0.01670838  0.1433759   0.23073971
 -0.077119   -0.0401081  -0.28365314  0.02109051 -0.11749142  0.30782583
 -0.11777149  0.2389059   0.00447267 -0.18783441  0.34949175  0.11220213
 -0.08014097  0.17790945  0.13828596 -0.13944112]


Y búscamos un documento similiar a la que pasamos por parámetro

In [ ]:
d2v_model.most_similar([Analyze], topn = 2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('leys', 0.9113891124725342), ('boyang', 0.9109729528427124)]